In [1]:
# train_model.py
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import joblib

# 1. Veriyi Yükle
df = pd.read_csv("airline-passengers.csv")
df['Month'] = pd.to_datetime(df['Month'])
df.set_index('Month', inplace=True)

# 2. Veri Ön İşleme
data = df.values.astype('float32')

# Normalizasyon (LSTM için kritik: veriyi 0-1 arasına çekmek)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Eğitim/Test Ayrımı (%67 Eğitim, %33 Test)
train_size = int(len(scaled_data) * 0.67)
test_size = len(scaled_data) - train_size
train_data, test_data = scaled_data[0:train_size,:], scaled_data[train_size:len(scaled_data),:]

# Veri Seti Oluşturma Fonksiyonu (Lookback)
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

# Geçmiş 3 aya bakarak 1 ay sonrasını tahmin et
look_back = 3
X_train, y_train = create_dataset(train_data, look_back)
X_test, y_test = create_dataset(test_data, look_back)

# LSTM için girdiyi şekillendir [Samples, Time Steps, Features]
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# 3. LSTM Modeli
print("LSTM Modeli Kuruluyor...")
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

# 4. Eğitim
print("Eğitim Başlıyor...")
model.fit(X_train, y_train, epochs=100, batch_size=1, verbose=2)

# 5. Kaydetme
model.save("lstm_airline_model.h5")
joblib.dump(scaler, "airline_scaler.pkl")
print("Model ve Scaler kaydedildi.")

C:\Users\wfi-7\anaconda3\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):
C:\Users\wfi-7\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


LSTM Modeli Kuruluyor...
Eğitim Başlıyor...
Epoch 1/100
92/92 - 1s - 16ms/step - loss: 0.0345
Epoch 2/100
92/92 - 0s - 1ms/step - loss: 0.0133
Epoch 3/100
92/92 - 0s - 1ms/step - loss: 0.0088
Epoch 4/100
92/92 - 0s - 1ms/step - loss: 0.0078
Epoch 5/100
92/92 - 0s - 1ms/step - loss: 0.0069
Epoch 6/100
92/92 - 0s - 1ms/step - loss: 0.0061
Epoch 7/100
92/92 - 0s - 1ms/step - loss: 0.0055
Epoch 8/100
92/92 - 0s - 1ms/step - loss: 0.0051
Epoch 9/100
92/92 - 0s - 1ms/step - loss: 0.0047
Epoch 10/100
92/92 - 0s - 1ms/step - loss: 0.0044
Epoch 11/100
92/92 - 0s - 1ms/step - loss: 0.0042
Epoch 12/100
92/92 - 0s - 2ms/step - loss: 0.0041
Epoch 13/100
92/92 - 0s - 1ms/step - loss: 0.0039
Epoch 14/100
92/92 - 0s - 1ms/step - loss: 0.0038
Epoch 15/100
92/92 - 0s - 1ms/step - loss: 0.0038
Epoch 16/100
92/92 - 0s - 1ms/step - loss: 0.0037
Epoch 17/100
92/92 - 0s - 2ms/step - loss: 0.0037
Epoch 18/100
92/92 - 0s - 1ms/step - loss: 0.0037
Epoch 19/100
92/92 - 0s - 1ms/step - loss: 0.0036
Epoch 20/100
9

Model ve Scaler kaydedildi.
